# Temporary Notebook for developing new Data prep Process


In [ ]:
from collections import defaultdict
from collections import Counter

import pandas as pd
from shapely.geometry import Polygon


In [ ]:
# Local functions and classes
from types_and_classes import *
from utilities import *
from contours import *
from contour_graph import *
from debug_tools import *
#from structures import *
#from region_slice import *
#from structure_set import *
#from relations import *


### Global Settings

In [ ]:
PRECISION = 2

In [ ]:
%matplotlib inline

### Convert contour points to polygons and sort them by area


1. Start with list of dictionaries of contour coordinates 
    - Each dictionary contains the following items: 
        - ROI number 
            - Int ROI reference 
        - SliceIndex 
            - Float Z coordinate 
        - Coordinates 
            - List of coordinate points that form a closed polygon. 
2. Generate a contours DataFrame from the dictionary 
3. Convert each list of coordinate points to a polygon and validate it. 
4. Sort the contours DataFrame rows by  
    - ROI 
    - SliceIndex 
    - Polygon area (decreasing) 

**SliceSequence** 

- Ordered list of all slice indexes in use 
- This sequence represents only the original contour slice indexes 
- **INIT**(Series of SliceIndex) 
    1. Take the SliceIndex column from the contours DataFrame  
    1. Remove duplicates 
    1. Sort (increasing) 
    1. Set SliceIndex as the DataFrame index, but keep it as a column 
    1. Combine as a 3-column DataFrame 
        - ThisSlice:  SliceIndex,  
        - NextSlice: SliceIndex shifted by + 1,  
        - PreviousSlice: SliceIndex shifted by - 1 
    1. Add a column 'Original' and set all values to True. 
- **get_neighbors**(SliceIndex) 
    > Return: 
    > - (*NextSlice*, *PreviousSlice*) for a given SliceIndex,  
    > - *None* if the  given SliceIndex is not in the sequence 

### Create identify contours that are holes and build a graph of the contours

#### Create an graph of the contours for the ROI

**Find Contour Neighbours**

- For each Node in the ROI-specific Contour Graph, *starting with the first SliceIndex*:

1. Use the SliceSequence to obtain the Next SliceIndex.
2. Find all Contour nodes of the same hole type *(IsHole True or IsHole False)* with that Next SliceIndex.
3. If the Hulls of the two polygons overlap:
    > 1. Create an edge between the two Contour nodes
    > 2. Set a contourMatch attribute to the edge with the two Contour nodes as values.
    > 3. Set the Thickness of the edge to the difference between the two SliceIndexes.

4. For each Node in the ROI-specific ContourGraph calculate a Thickness from its edges.

**Contour Boundaries**
1. Select all Contour nodes in the ContourGraph with only one edge (degree=1)
2. For each of these nodes:
    1. Generate an interpolated boundary Contour 
        > - The contour is located half way between the boundary contour slice 
            and the neighbour slice that is not linked with an edge.
        > - set IsInterpolated to True
        > - set IsBoundary is True
        > - set IsHole matches the IsHole value of the non-interpolated Contour 
    1. Add the interpolated SliceIndex to SliceSequence:
        > - set 'Original' = False
        > - If the boundary contour SliceIndex  < the interpolated SliceIndex 
            >> - set 'PreviousSlice' = boundary contour SliceIndex, 
            >> - set NextSlice'= None
        > - Else:
            >> - set 'NextSlice'= boundary contour SliceIndex, 
            >> - set 'PreviousSlice '= None
    1. Add the interpolated contour to the contour graph .
    1. Add a ContourMatch edge between the original boundary contour and the interpolated contour.

**EnclosedRegionGraph**
- SubGraph of ContourGraph
- Identified as all nodes in ContourGraph that have a path between them
- Has a unique RegionIndex


**EnclosedRegions**
1. Create an EnclosedRegion SubGraph for each collection of nodes in the ContourGraph that are connected by a path.
2. For each EnclosedRegion:
    1. Create a new EnclosedRegionLabel.
    2. For each Contour node in the EnclosedRegion.
        1. Set Contour.RegionIndex to the EnclosedRegionLabel.
        2. Increment and Identify and label continuous contour regions.
        
**Note:** *Changes to the Contour nodes must reflect in the nodes of the ContourGraph*


**Open (External) Holes**
1. For each EnclosedRegion that is a hole:
    1. Select its boundary contours.
    2. Check whether there is a non-hole contour on the next slice that completely contains the hole boundary.
    3. If any hole boundary does not have a non-hole contour on the next slice that completely contains that hole boundary, then the hole is ***Open*** (external).
    4. If all hole boundaries have a non-hole contour on the next slice that completely contains that hole boundary, then the hole is ***Closed*** (internal).



### RegionSlices


**RegionSlice**

RegionSlice is generated by adding all contours from the same EnclosedRegion and SliceIndex, subtracting all holes.  

  - Related External holes are stored as a separate MultiPolygon attribute.
  - RegionIndex
  - SliceIndex
  - Polygon
  - ExternalHoles
  - Boundaries
  - Thickness
  - Contours
      - List of associated ContourIndexes

**INIT**

1. Receive all Contours from an EnclosedRegion with that SliceIndex.
1. Populate the Contours attribute with a list of the node labels for the supplied Contours.
1. Create a MultiPolygon by combining the Contour polygons that are not holes and not boundaries.
1. Subtract the Contour polygons that are holes from the MultiPolygon
1. Store the MultiPolygon in the Polygon attribute.
1. Create another MultiPolygon by combining all boundary contours and subtracting all hole boundaries. store this in the Boundaries attribute.
1. Create a third MultiPolygon by combining all External holes and store this in the ExternalHoles attribute.
        


**EnclosedRegionTable**

- RegionIndex
- SliceIndex
- RegionSlice


1. Create an EnclosedRegionTable
2. For each SliceIndex in each EnclosedRegion: 
    1. Create a RegionSlice using all Contours from the EnclosedRegion with that SliceIndex
    2. Calculate an approximate thickness value and set the RegionSlice Thickness attribute.
        - Thickness = sum(edge.thickness*edge.combined_area)/sum(edge.combined_area)
    3. Add the RegionSlice to the EnclosedRegionTable 
3. Place the EnclosedRegionTable and ContourGraph in a Structure


### Structures 

For each ROI: 

1. Create a ContourGraph 
2. Create an EnclosedRegionTable
3. Create a Structure 


**Structure**

*Attributes:*

  - ROI Number 
  - ID (name)
  - ContourGraph 
  - EnclosedRegionTable 
  - Physical volume 
  - External volume 
  - Hull volume 

**Node Volume**
- The volume of a node is approximated by a set of pseudo-cylinders
- Each pseudo-cylinder is height is half of the gap between the node and one of 
   its neighbours.
- the volume of the cylinder is the area of the cylinder times the half of the 
  gap.
- If the node has only one neighbour in a given direction (positive or negative), 
  then the area of the cylinder is the weighted average of the area of the node 
  and its neighbour.
  - The weight of the node area is 0.75 and the weight of the neighbouring 
    area is 0.25.
  - These weights are used to approximate the volume of the node as a cylinder 
     in the region that is half of the gap between the node and the neighbour.
- If the node has only one neighbour in each direction and the same gap, then 
  the volume of the node becomes the volume of a cylinder with the height equal 
  to the gap and area as 0.75 times the area of the node and 0.25 times the 
  area of the neighbouring nodes.
  > $\bar{A_i} = \frac{3}{4}A_N + \frac{1}{4}A_i$

- If the node has more than one neighbour in a given direction, then the volume of the 
  node is approximated by the sum of the volumes of the pseudo-cylinders 
  between the node and its neighbours.
- The sum of the areas of the pseudo-cylinders will be between the area of the 
  node and the area of the neighbouring nodes.
  > $\bar{A_i} = \frac{3\tilde{A_N} + A_i}{4}$<br><br>
  > $\tilde{A_N} = A_n\frac{A_i}{\sum{A_i}}$<br><br>
  > $\bar{A_i} = \frac{A_i}{4}\left(\frac{3A_N}{\sum{A_i}} + 1\right)$

- If the node has only one neighbour in a given direction, the weighting factor becomes:
  > $\bar{A_i} = \frac{A_i}{4}\left(\frac{3A_N}{A_i} + 1\right)$<br><br>
  > $\bar{A_i} = \frac{3}{4}A_N + \frac{1}{4}A_i$
  > Which agrees with the previous case.

The *combined area* ($A_c$) of an edge is the sum of the area of both nodes on the edge.  Rewriting the equation for the weighted area in terms of $A_N$ and $A_c$ gives:
  > $A_c = A_N + A_i$<br><br>
  > $\sum{A_i} = \sum{A_c} - n_iA_N$<br><br>
  > $\bar{A_i} = \frac{A_i}{4}\left(\frac{3A_N}{\sum{A_c} - n_iA_N} + 1\right)$<br><br>
  > $\bar{A_i} = \frac{A_i}{4}\left(\frac{3}{\sum{\frac{A_c}{A_N}} - n_i} + 1\right)$<br><br>

- The volume of the node is the sum of the volumes of the pseudo-cylinders multiplied by the thickness of the corresponding edges.
  
> Note: There is a difference between averaging by radius (r) and averaging by area ($r^2$) and averaging by volume ($r^3$).


**Calculate Structure volumes**

- **Physical** 
    1. For each node in ContourGraph: 
        1. Get the pseudo-area of the region between the node and each of its neighbours.
        2. Calculate a slice volume as area * thickness 
        3. If the node is a hole set the volume as negative 
    1. Sum the slice volumes 

- **Exterior** 
    1. For each node in ContourGraph: 
        1. Get the area of the polygon 
        1. Get an approximate thickness value (based on all edges connected to the node). 
            - Thickness = sum(edge.thickness*edge.combined_area)/sum(edge.combined_area) 
        1. Calculate a slice volume as area * thickness 
        1. If the node is an open (exterior) hole set the volume as negative 
        1. If the node is a closed (interior) hole set the volume to zero
    1. Sum the slice volumes 

- **Hull** 
    1. For each node in ContourGraph: 
        1. Get the area of the convex hull of the Polygon
        1. Calculate a slice volume as area * thickness 
        1. If the node is a hole set the volume to zero 
    1. Sum the slice volumes

 

### Create interpolated slices as needed for each contour graph
1. Find all slice indexes that are not original
2. For each slice index:
    1. use contour_lookup to determine if that slice exists in the contour graph.'
    2. If it does not exist, but the previous and next slices do, and have the same region index:
        1. create an interpolated contour at the specified index.
        2. set the IsInterpolated attribute to True.
        3. set the region index to the region index of the previous slice.
        4. set the IsBoundary attribute to False
        4. add the interpolated contour to the contour graph.
        5. Add edges to the contour graph between the interpolated contour and the previous and next slices.


## Data Processing Tests

- use the hole_test_contour_table function since it is the most complete.
- Check the data in the columns
- ROI, SliceIndex, HoleType, Interpolated, Boundary,  RegionIndex,
- and Label
- ROI should have the same value for the entire column
- SliceIndex should be sorted in ascending order. It should have values
- from 0 to 5 and half integer values for the boundaries
- Contours that are boundaries should also be interpolated.
- The hole type should be "None" for all contours that are not holes.
- Label should be a tuple of the ROI, the SliceIndex and the ContourIndex.
- ROI 1 should have boundaries on slice 0.5 and 4.5,
- the hole region should be "Open" at both ends.
- It should have two regions.
- ROI 2 should have boundaries on slice 0.5, 1.5, 3.5 and 4.5,
- the hole region should be "Closed".
- It should have two regions.

In [ ]:
def single_box_example():
    slice_spacing = 0.1
    # Body structure defines slices in use
    box3 = make_box(roi_num=1, width=3, spacing=slice_spacing)
        # combine the contours
    slice_data = box3
    return slice_data

- Box Width = 3
- Slice area = 9
- Slice Spacing = 0.1

- Volume of each node = 3 * 3 * 0.1 = 0.9

Ideal volume is $3 \times 3 \times 3 = 27$

Actual volume should be  a bit less than $3 \times 3 \times (3 + 0.1) = 27.9$

- The extra volume is due to the 1/2 slice projection of the box in the Z direction
- but is reduced slightly due to interpolation of the slice area

In [ ]:
def box_with_hole_example():
    slice_spacing = 0.1
    # Body structure defines slices in use
    box6 = make_box(roi_num=1, width=6, spacing=slice_spacing)
    box3 = make_box(roi_num=1, width=3, spacing=slice_spacing)
    # combine the contours
    slice_data = box6 + box3
    return slice_data

- Box Width = 6
- Slice area = 6x6 = 36
- Hole Width = 3
- Hole area = 3 x 3 = 9
- Slice Spacing = 0.1

- Volume of each node = (6 * 6 - 3 * 3) * 0.1 = (36 - 9) * 0.1 = 2.7

- Ideal volume is $6 \times 6 \times 6 - 3 \times 3 \times 3 = 216 - 27 = 189$

- Actual volume should be a bit less than 
$6 \times 6 \times (6 + 0.1) - 3 \times 3 \times (3 + 0.1) = 219.6 - 27.9 = 191.7$

> - The extra volume is due to the 1/2 slice projection of the box in the Z direction
> - but is reduced slightly due to interpolation of the slice area

- External volume is the same as the ideal volume, but without the holes subtracted.
- External volume should be a bit less than 
$6 \times 6 \times (6 + 0.1) = 219.6$


In [ ]:
def two_boxes_example():
    slice_spacing = 0.1
    # Body structure defines slices in use
    box3_left = make_box(roi_num=1, width=3, offset_x=-2, spacing=slice_spacing)
    box3_right = make_box(roi_num=1, width=3, offset_x=2, spacing=slice_spacing)
    # combine the contours
    slice_data = box3_left + box3_right
    return slice_data

In [ ]:
def square_tube_example():
    slice_spacing = 0.1

    # Body structure defines slices in use
    box6 = make_box(roi_num=1, width=6, spacing=slice_spacing)
    box3 = make_box(roi_num=1, width=3, height=6,spacing=slice_spacing)
    # combine the contours
    slice_data = box6 + box3
    return slice_data

In [ ]:
def embedded_boxes_example():
    slice_spacing = 0.5
    # Body structure defines slices in use
    box6 = make_box(roi_num=1, width=6, spacing=slice_spacing)
    box3 = make_box(roi_num=2, width=3, spacing=slice_spacing)
    # combine the contours
    slice_data = box6 + box3
    return slice_data

In [ ]:
def hollow_horizontal_cylinder_example():
    slice_spacing = 1
    # Body structure defines slices in use
    body = make_vertical_cylinder(roi_num=0, radius=12, length=16, offset_z=0,
                                  spacing=slice_spacing)
    outer_cylinder = make_horizontal_cylinder(roi_num=1, radius=6, length=10,
                                              spacing=slice_spacing)
    cylinder_hole = make_horizontal_cylinder(roi_num=1, radius=4, length=10,
                                             spacing=slice_spacing)
    # combine the contours
    slice_data = outer_cylinder + cylinder_hole + body
    return slice_data


In [ ]:
def embedded_spheres_example():
    slice_spacing = 0.5
    # Body structure defines slices in use
    sphere6 = make_sphere(roi_num=1, radius=3, spacing=slice_spacing)
    sphere3 = make_sphere(roi_num=2, radius=1.5, spacing=slice_spacing)
    # combine the contours
    slice_data = sphere6 + sphere3
    return slice_data

In [ ]:
def multiple_spheres_example():
    slice_spacing = 0.5
    body = make_sphere(roi_num=0, radius=3, spacing=slice_spacing)
    left_sphere = make_sphere(roi_num=1, radius=2, offset_x=-2.5, offset_z=0,
                              spacing=slice_spacing)
    right_sphere = make_sphere(roi_num=1, radius=2, offset_x=2.5, offset_z=0,
                               spacing=slice_spacing)
    right_middle_sphere = make_sphere(roi_num=2, radius=1, offset_x=2.5, offset_z=0,
                                    spacing=slice_spacing)
    # combine the contours
    slice_data = left_sphere + right_sphere + right_middle_sphere + body
    return slice_data

In [ ]:
def surrounded_horizontal_cylinder_example():
    slice_spacing = 1
    # Body structure defines slices in use
    body = make_vertical_cylinder(roi_num=0, radius=12, length=16, offset_z=0,
                                  spacing=slice_spacing)
    outer_cylinder = make_horizontal_cylinder(roi_num=1, radius=6, length=10,
                                              spacing=slice_spacing)
    cylinder_hole = make_horizontal_cylinder(roi_num=1, radius=5, length=8,
                                             spacing=slice_spacing)
    surrounded_cylinder = make_horizontal_cylinder(roi_num=2, radius=3, length=6,
                                                   spacing=slice_spacing)
    # combine the contours
    slice_data = body + outer_cylinder + cylinder_hole + surrounded_cylinder
    # convert contour slice data into a table of slices and structures
    return slice_data

In [ ]:
def shelters_horizontal_cylinder_single_side_example():
    slice_spacing = 1
    # Body structure defines slices in use
    body = make_vertical_cylinder(roi_num=0, radius=12, length=16, offset_z=0,
                                  spacing=slice_spacing)
    outer_cylinder = make_horizontal_cylinder(roi_num=1, radius=6, length=10,
                                              spacing=slice_spacing)
    cylinder_hole = make_horizontal_cylinder(roi_num=1, radius=4, length=8,
                                             offset_x=1, offset_z=0,
                                             spacing=slice_spacing)
    surrounded_cylinder = make_horizontal_cylinder(roi_num=2, radius=3,
                                                   length=6, offset_x=1,
                                                   spacing=slice_spacing)
    # combine the contours
    slice_data = outer_cylinder + cylinder_hole + surrounded_cylinder + body
    return slice_data


In [ ]:
def sphere_in_shell_example():
    slice_spacing = 1
    # Body structure defines slices in use
    body = make_vertical_cylinder(roi_num=0, radius=16, length=20, offset_z=0,
                                    spacing=slice_spacing)
    sphere12 = make_sphere(roi_num=1, radius=6, spacing=slice_spacing)
    hole10 = make_sphere(roi_num=1, radius=5, spacing=slice_spacing)
    sphere6 = make_sphere(roi_num=2, radius=3, spacing=slice_spacing)

    # combine the contours
    slice_data = body + sphere12 + hole10 + sphere6
    return slice_data

In [ ]:
def hole_test_contour_table():
    '''Create a contour table for testing set_enclosed_regions function.

    The test table contains the following data:
        ROI 0: The background region that defines the set of all slice indexes.
            slices, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0
        ROI 1: 1 contour with an embedded hole on slices 1.0 to 4.0,
            results in open hole
        ROI 2: 1 contour on slices 1.0 to 4.0,
               an embedded hole on slices 2.0 and 3.0,
               results in a closed hole.
        ROI 3: 1 contour on slices 1.0 to 4.0,
               an embedded hole on slices 1.0 and 3.0,
               results in a open hole.
    '''
    box1= box_points(width=1)
    box3_right = box_points(width=3, offset_x=-1.5)
    box1_right = box_points(width=1, offset_x=-1.5)
    box3_left = box_points(width=3, offset_x=1.5)
    box1_left = box_points(width=1, offset_x=1.5)
    box3_ant = box_points(width=3, offset_y=1.5)
    box1_ant = box_points(width=1, offset_y=1.5)
    slice_data = [
       ContourPoints(box1, roi=0, slice_index=0.0),        # ROI 0, Area 1
       ContourPoints(box1, roi=0, slice_index=1.0),        # ROI 0, Area 1
       ContourPoints(box1, roi=0, slice_index=2.0),        # ROI 0, Area 1
       ContourPoints(box1, roi=0, slice_index=3.0),        # ROI 0, Area 1
       ContourPoints(box1, roi=0, slice_index=4.0),        # ROI 0, Area 1
       ContourPoints(box1, roi=0, slice_index=5.0),        # ROI 0, Area 1

       ContourPoints(box3_right, roi=1, slice_index=1.0),  # ROI 1, Area 4
       ContourPoints(box1_right, roi=1, slice_index=1.0),  # ROI 1, Hole
       ContourPoints(box3_right, roi=1, slice_index=2.0),  # ROI 1, Area 4
       ContourPoints(box1_right, roi=1, slice_index=2.0),  # ROI 1, Hole
       ContourPoints(box3_right, roi=1, slice_index=3.0),  # ROI 1, Area 4
       ContourPoints(box1_right, roi=1, slice_index=3.0),  # ROI 1, Hole
       ContourPoints(box3_right, roi=1, slice_index=4.0),  # ROI 1, Area 4
       ContourPoints(box1_right, roi=1, slice_index=4.0),  # ROI 1, Hole

       ContourPoints(box3_left, roi=2, slice_index=1.0),  # ROI 2, Area 4
       ContourPoints(box3_left, roi=2, slice_index=2.0),  # ROI 2, Area 4
       ContourPoints(box1_left, roi=2, slice_index=2.0),  # ROI 2, Hole
       ContourPoints(box3_left, roi=2, slice_index=3.0),  # ROI 2, Area 4
       ContourPoints(box1_left, roi=2, slice_index=3.0),  # ROI 2, Hole
       ContourPoints(box3_left, roi=2, slice_index=4.0),  # ROI 2, Area 4

       ContourPoints(box3_ant, roi=3, slice_index=1.0),  # ROI 3, Area 4
       ContourPoints(box1_ant, roi=3, slice_index=1.0),  # ROI 3, Hole
       ContourPoints(box3_ant, roi=3, slice_index=2.0),  # ROI 3, Area 4
       ContourPoints(box1_ant, roi=3, slice_index=2.0),  # ROI 3, Hole
       ContourPoints(box3_ant, roi=3, slice_index=3.0),  # ROI 3, Area 4
       ContourPoints(box1_ant, roi=3, slice_index=3.0),  # ROI 3, Hole
       ContourPoints(box3_ant, roi=3, slice_index=4.0),  # ROI 3, Area 4
    ]
    contour_table, slice_sequence = build_contour_table(slice_data)
    return contour_table, slice_sequence


## Test RegionSlice

## Apply Data Preprocess to the data

In [ ]:
slice_data = single_box_example()
contour_table, slice_sequence = build_contour_table(slice_data)
contour_graph, slice_sequence = build_contour_graph(contour_table,
                                                    slice_sequence,
                                                    roi=1)
Counter(dict(contour_graph.degree).values())

In [ ]:
contour_table, slice_sequence = hole_test_contour_table()
contour_graph, slice_sequence = build_contour_graph(contour_table,
                                                    slice_sequence,
                                                    roi=0)
degrees = Counter(dict(contour_graph.degree).values())
degrees

In [ ]:
contour_table, slice_sequence = hole_test_contour_table()
contour_graph, slice_sequence = build_contour_graph(contour_table,
                                                    slice_sequence,
                                                    roi=3)
degrees = Counter(dict(contour_graph.degree).values())
degrees

In [ ]:
nx.draw(contour_graph, with_labels=True)
plt.show()

## Testing RegionSlice

### Test Preprocessing
1. define the contours for the slices
2  Build the contour graph

Create RegionSlices for different slices

- Items to test:
    - regions, open_holes, exterior, hull, region_holes, embedded_regions
        1. region for slice with one contour
            - region contains a single MultiPolygon
            - boundaries contains an empty MultiPolygon
            - open_holes contains an empty MultiPolygon
            - exterior is the same as the region
            - hull is the same as the region
        2. region for slice with two non-overlapping contours
            - region contains two MultiPolygons
            - boundaries contains two empty MultiPolygons
            - open_holes contains two empty MultiPolygons
            - exterior is the same as the region
            - hull is the same as the region
        2. region for slice with two non-overlapping contours that are part of the same region
            - region contains a single MultiPolygon that includes the two contours
            - boundaries contains an empty MultiPolygon
            - open_holes contains an empty MultiPolygon
            - exterior is the same as the region
            - hull is the convex hull of the region MultiPolygon
        4. region for slice with contour and an open hole
            - region contains a single MultiPolygon
            - boundaries contains an empty MultiPolygon
            - open_holes contains the hole as a single MultiPolygon
            - exterior is the same as the region
            - hull contains the larger contour as a single MultiPolygon            
        3. region for slice with contour and a closed hole
            - region contains a single MultiPolygon
            - boundaries contains an empty MultiPolygon
            - open_holes contains an empty MultiPolygon
            - exterior contains the larger contour as a single MultiPolygon
            - hull is the convex hull of the region MultiPolygon
        4. region for slice with contour and open hole
            - region contains a single MultiPolygon
            - boundaries contains an empty MultiPolygon
            - open_holes contains the hole as a single MultiPolygon
            - exterior is the same as the region
            - hull is the convex hull of the region MultiPolygon

    - boundaries, is_interpolated
        1. boundary slice for region with one contour per slice
            - boundaries contains a single MultiPolygon
            - region contains an empty MultiPolygon
            - open_holes contains an empty MultiPolygon
            - is_interpolated is True
        2. boundary slice for one of two regions  **???? Is this possible? ????**
            - boundaries contains two MultiPolygons, one of which is empty
            - region contains two MultiPolygons, one of which is empty
            - open_holes contains two empty MultiPolygons
            - is_interpolated is False

    - contour_indexes

    - is_empty
    - applied boolean



In [ ]:
slice_data = single_box_example()
contour_table, slice_sequence = build_contour_table(slice_data)
outer_box = StructureShape(roi=1, name='outer_box')
slice_sequence = outer_box.build_contour_graph(contour_table, slice_sequence)

inner_box = StructureShape(roi=2, name='inner_box')
slice_sequence = inner_box.build_contour_graph(contour_table, slice_sequence)


In [ ]:
not_original = slice_sequence.sequence.Original == False
interpolated_slice_indexes = list(slice_sequence.sequence.loc[not_original,
                                                              'ThisSlice'])
outer_box.generate_interpolated_contours(slice_sequence,
                                         interpolated_slice_indexes)
inner_box.generate_interpolated_contours(slice_sequence,
                                         interpolated_slice_indexes)

In [ ]:
outer_box.contour_lookup.columns


In [ ]:
inner_box.contour_lookup

In [ ]:
slice_data = single_box_example()
contour_table, slice_sequence = build_contour_table(slice_data)

roi = 1
contour_graph, slice_sequence = build_contour_graph(contour_table,
                                                    slice_sequence, roi)
contour_lookup = build_contour_lookup(contour_graph)

#enclosed_region_table = build_region_table(contour_graph, contour_lookup)

In [ ]:
nx.draw(contour_graph, with_labels=True)

In [ ]:
slice_data = box_with_hole_example()
contour_table, slice_sequence = build_contour_table(slice_data)
roi = 1
contour_graph, slice_sequence = build_contour_graph(contour_table,
                                                    slice_sequence, roi)
contour_lookup = build_contour_lookup(contour_graph)
enclosed_region_table = build_region_table(contour_graph, contour_lookup)

In [ ]:
slice_data = square_tube_example()
contour_table, slice_sequence = build_contour_table(slice_data)
roi = 1
contour_graph, slice_sequence = build_contour_graph(contour_table,
                                                    slice_sequence, roi)
contour_lookup = build_contour_lookup(contour_graph)

enclosed_region_table = build_region_table(contour_graph, contour_lookup)

## Relation analysis process 
1. Sort structures by increasing Hull volume 
2. Starting with largest, get relations for all other structures 
    1. for each slice containing the structure:
        1. for each RegionSlice of that structure on the slice:
            1. Get a 27 bit relation with all RegionSlices of the other 
               structure that are on the same slice. 
                - MultiPolygon vs MultiPolygon
                - (BoundaryPolygon + External holes) vs MultiPolygon  
                - Convex Hull  vs MultiPolygon 
            1. Apply appropriate corrections for holes and boundaries. 
            1. If the region is a boundary, build a temporary RegionSlice from ContourGraph 
                1. If there is no matching slice for the other structure, 
                    interpolate the closest slices in the regions of the other 
                    structure. 
                1. Use SliceSequence to find nearest possible slices 
                1. If boundary slice exists in SliceSequence then no other structure 
                1. Get two nearest slice index 
                1. Find all Contours for other structure at those slice indexes 
                1. Interpolate contour points onto boundary slice 
                1. Generate new interpolated contours 
                1. Add interpolated contours to ContourGraph, but do not update SliceSequence  
                1. *Create a new RegionSlice?*
    2. Combine all relations with OR 
2. Create a relations Graph with structures as nodes and relations as edges 
2. Identify Logical relationships 

**StructureSet**

- List of Structures 
- RelationshipGraph 